In [15]:
from fiona.transform import transform_geom
from fiona.crs import from_epsg
from statistics import mean 
import fiona
from pyproj import Proj
from shapely.geometry import shape
# split files based on OSM or not
# make sure all geometries are valid or reject them
# get total length/area of files
publicland = ['publicland_aggregated.shp', 'publicland_rejected.shp']
trails = ['trails_rejected.shp', 'trails_aggregated.shp']

for f in publicland:
    with fiona.open(f, 'r') as fin:
        origschema = fin.schema
        newfeats_auth = []
        newfeats_osm = []
        osm_area = []
        auth_area = []
        for feat in fin:
            feat['geometry'] = transform_geom(str(Proj(init='EPSG:4326').crs), str(Proj(init='EPSG:3857').crs), feat['geometry'])
            if feat['properties']['OManaging'] == 'OSM':
                newfeats_osm.append(feat)
                osm_area.append(shape(feat['geometry']).area)
            else:
                newfeats_auth.append(feat)
                auth_area.append(shape(feat['geometry']).area)

    with fiona.open(f.replace('.shp', 'prof_osm.shp'), 'w', driver='ESRI Shapefile', schema=origschema, crs=from_epsg(3857)) as fout:
        for feat in newfeats_osm:
            fout.write(feat)
        print(f.replace('.shp', 'prof_osm.shp'), ":", "sum:", sum(osm_area)/1000000, "sq km", "mean:", mean(osm_area)/1000000 )
    with fiona.open(f.replace('.shp', 'prof_auth.shp'), 'w', driver='ESRI Shapefile', schema=origschema, crs=from_epsg(3857)) as fout:
        for feat in newfeats_auth:
            fout.write(feat)
        print(f.replace('.shp', 'prof_auth.shp'), ":", "sum:", sum(auth_area)/1000000, "sq km", "mean:", mean(auth_area)/1000000 )

publicland_aggregatedprof_osm.shp : sum: 4395.854962761126 sq km mean: 7.963505367320868
publicland_aggregatedprof_auth.shp : sum: 9688.560516583364 sq km mean: 4.165331262503595
publicland_rejectedprof_osm.shp : sum: 443.5044381851666 sq km mean: 2.563609469278419
publicland_rejectedprof_auth.shp : sum: 2442.406839441378 sq km mean: 6.918999545159712


In [16]:
for f in trails:
    with fiona.open(f, 'r') as fin:
        origschema = fin.schema
        newfeats_auth = []
        newfeats_osm = []
        osm_area = []
        auth_area = []
        for feat in fin:
            feat['geometry'] = transform_geom(str(Proj(init='EPSG:4326').crs), str(Proj(init='EPSG:3857').crs), feat['geometry'])
            if feat['properties']['OManaging'] == 'OSM':
                newfeats_osm.append(feat)
                osm_area.append(shape(feat['geometry']).length)
            else:
                newfeats_auth.append(feat)
                auth_area.append(shape(feat['geometry']).length)

    with fiona.open(f.replace('.shp', 'prof_osm.shp'), 'w', driver='ESRI Shapefile', schema=origschema, crs=from_epsg(3857)) as fout:
        for feat in newfeats_osm:
            fout.write(feat)
        print(f.replace('.shp', 'prof_osm.shp'), ":", "sum:", sum(osm_area)/1000, "km", "mean:", mean(osm_area)/1000 )
    with fiona.open(f.replace('.shp', 'prof_auth.shp'), 'w', driver='ESRI Shapefile', schema=origschema, crs=from_epsg(3857)) as fout:
        for feat in newfeats_auth:
            fout.write(feat)
        print(f.replace('.shp', 'prof_auth.shp'), ":", "sum:", sum(auth_area)/1000, "km", "mean:", mean(auth_area)/1000 )

trails_rejectedprof_osm.shp : sum: 1030.893281426378 km mean: 1.3016329310939116
trails_rejectedprof_auth.shp : sum: 616.5318693057761 km mean: 4.8931100738553654
trails_aggregatedprof_osm.shp : sum: 2776.017748979176 km mean: 0.4753455049621852
trails_aggregatedprof_auth.shp : sum: 3119.5491517778837 km mean: 2.3777051461721683
